# Calibration of motor current

In [ ]:
%matplotlib notebook

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

from tools.data_loader import moving_average

import glob
import os

In [ ]:
class Motor:
    
    def __init__(self, motorNumber):
        
        self.motorNumber = motorNumber
        self.currentESC = self.get_ESC_current(motorNumber)
        self.voltageESC = self.get_ESC_voltage(motorNumber)
        
        # get data and power setting from multimeter
        
        voltageDataMultimeter = self.get_multimeter_voltage(motorNumber)
        
        self.voltageMultimeter = voltageDataMultimeter.iloc[motorNumber-1].values
        self.powerSettingVoltage = [int(v[1:]) for v in voltageDataMultimeter.columns]
        
        currentDataMultimeter = self.get_multimeter_current(motorNumber)
        
        self.currentMultimeter = currentDataMultimeter.iloc[0].values
        self.powerSettingCurrent = np.array([int(v[1:]) for v in currentDataMultimeter.columns])
        
        # get time, RPM, voltage and current
        
        self.timeCurrent = ((self.currentESC['t'].values - min(self.currentESC['t'].values)) / 10e5)
        self.timeVoltage = ((self.voltageESC['t'].values - min(self.voltageESC['t'].values)) / 10e5)

        self.omegaVoltage = self.voltageESC[f'omega{motorNumber}'].values
        self.omegaCurrent = self.currentESC[f'omega{motorNumber}'].values

        self.voltage = self.voltageESC[f'U1{motorNumber}'].values
        self.current = self.currentESC[f'I1{motorNumber}'].values
        
        self.voltageSequences = self.get_voltage_sequence()
        self.currentSequences = self.get_current_sequence()
        
        # perform regression for RPM
        self.throttleRange = np.linspace(20, 100, 100).reshape(-1,1)
        self.omegaRegressionCurve = self.rpm_regression()
        self.ESCcurrentRegressionCurve = self.esc_current_regression()
        self.multimeterCurrentRegressionCurve = self.multimeter_current_regression()
    
    @staticmethod
    def get_files_for_limit(limit, flight, discard=True):
    
        v_induced = 14.345219306215128
        v_limit = limit * 0.01 * v_induced
        data_directory = flight + f'hover_{limit}_limit'
        csv_files = glob.glob(os.path.join(data_directory, "*.csv"))
        sorted_files = []

        # sort sequences in ascending order
        for i in range(len(csv_files)):
            for e in csv_files:
                if f'sequence_{i}_' in e:
                    #print(e)
                    sorted_files.append(e)
                    break

        # discard first and land sequence (vehicle on ground before takeoff and after landing)
        if discard:
            return sorted_files[1:-1]

        else:
            return sorted_files
    
    
    def get_ESC_current(self, motorNumber):
        
        filepath = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/current_motor_{motorNumber}/' 
        filesCurrent = self.get_files_for_limit(10, filepath, discard=False)
        
        currentData = pd.DataFrame()
        
        for file in filesCurrent:
            currentData = pd.concat([currentData, pd.read_csv(file)])
        
        return currentData
    
    
    
    def get_ESC_voltage(self, motorNumber):
        
        filepath = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/voltage_motor_{motorNumber}/'
        filesVoltage = self.get_files_for_limit(10, filepath, discard=False)
        
        voltageData = pd.DataFrame()
        
        for file in filesVoltage:
            voltageData = pd.concat([voltageData, pd.read_csv(file)])
        
        return voltageData
    
    
    @staticmethod
    def get_multimeter_voltage(motorNumber):
        
        filepath = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/voltage_data.csv'
        voltage_measurements = pd.read_csv(filepath)
        
        return voltage_measurements#.iloc[motorNumber-1].values

    
    @staticmethod
    def get_multimeter_current(motorNumber):
        
        folder = f'/home/gregor/Masterarbeit/flight_data/2022-08-08_test_rig/current_data_multimeter/'
        file = f'current_motor{motorNumber}.csv'
        
        current_measurement = pd.read_csv(folder+file)
        
        return current_measurement#.iloc[0].values
    
    
    def get_current_sequence(self):
        
        relevantRpmValues = np.unique(self.omegaCurrent[self.omegaCurrent>=1000], return_counts=True) 
        frequentRpmValues = relevantRpmValues[0][relevantRpmValues[1] > 20]
        filteredIndices = np.nonzero(np.isin(self.omegaCurrent, frequentRpmValues)==1)[0]
        
        startMask = np.where(np.diff(filteredIndices,prepend=0)>10) # search for the gaps between the sequences
        sequenceStart = filteredIndices[startMask]

        endMask = np.where(np.abs(np.diff(np.flip(filteredIndices),prepend=0))>10)
        sequenceEnd = np.flip(np.flip(filteredIndices)[endMask])
        
        filteredCurrentSequence = []
        
        for i in range(sequenceStart.size):
    
            intersection = np.intersect1d(np.arange(sequenceStart[i],sequenceEnd[i]), filteredIndices)
            
            # remove first and last 5 values to get rid of outliers
            if not intersection.size == 0:
                filteredCurrentSequence.append(intersection[5:-5])
            
        return filteredCurrentSequence
    
    
    def get_voltage_sequence(self):
        
        relevantRpmValues = np.unique(self.omegaVoltage[self.omegaVoltage>=1000], return_counts=True) 
        frequentRpmValues = relevantRpmValues[0][relevantRpmValues[1] > 20]
        filteredIndices = np.nonzero(np.isin(self.omegaVoltage, frequentRpmValues)==1)[0]
        
        startMask = np.where(np.diff(filteredIndices,prepend=0)>10) # search for the gaps between the sequences
        sequenceStart = filteredIndices[startMask]

        endMask = np.where(np.abs(np.diff(np.flip(filteredIndices),prepend=0))>10)
        sequenceEnd = np.flip(np.flip(filteredIndices)[endMask])
        
        filteredVoltageSequence = []
        
        for i in range(sequenceStart.size):
    
            intersection = np.intersect1d(np.arange(sequenceStart[i],sequenceEnd[i]), filteredIndices)
            
            # remove first and last 5 values to get rid of outliers
            if not intersection.size == 0:
                filteredVoltageSequence.append(intersection[5:-5])
            
        return filteredVoltageSequence
    
    def rpm_regression(self):
        
        throttleSetpoints = np.zeros(len(self.voltageSequences))
        meanOmega = np.zeros(len(self.voltageSequences))
        meanVoltage = np.zeros(len(self.voltageSequences))
        
        for i, seq in enumerate(self.voltageSequences):
            
            throttleSetpoints[i] = self.powerSettingVoltage[i]
            
            omega = self.omegaVoltage[seq]
            voltage = self.voltage[seq]
            
            meanOmega[i] = np.mean(omega)
            meanVoltage[i] = np.mean(voltage)
        
        #self.voltageBias = np.sum(meanVoltage[:4] - self.voltageMultimeter[:4])/4
        self.voltageBias = np.sum(meanVoltage - self.voltageMultimeter)/self.voltageMultimeter.size
        
        print(f'Motor {self.motorNumber}: voltageBias = {self.voltageBias} V')
        #print(f'Motor {self.motorNumber}: voltageBias2 = {self.voltageBias2} V')
        
        features = PolynomialFeatures(degree=2).fit_transform(throttleSetpoints.reshape(-1,1))
        omegaRegression = LinearRegression().fit(features, meanOmega)
        omegaCurve = omegaRegression.predict(PolynomialFeatures(degree=2).fit_transform(self.throttleRange))
    
        return omegaCurve
    
    def esc_current_regression(self):
        
        throttleSetpoints = np.zeros(len(self.currentSequences))
        meanCurrent = np.zeros(len(self.currentSequences))
        
        for i, seq in enumerate(self.currentSequences[:4]):
            
            throttleSetpoints[i] = self.powerSettingCurrent[i]
            
            current =  self.current[seq]
            meanCurrent[i] = np.mean(current)
        
        self.currentBias = np.sum(meanCurrent[:4] - self.currentMultimeter[:4])/4
        
        #print(f'Motor {self.motorNumber}: currentBias = {self.currentBias} A')
        
        features = PolynomialFeatures(degree=3).fit_transform(throttleSetpoints.reshape(-1,1))
        currentRegression = LinearRegression().fit(features, meanCurrent-self.currentBias)
        currentCurve = currentRegression.predict(PolynomialFeatures(degree=3).fit_transform(self.throttleRange))
    
        return currentCurve
    
    def multimeter_current_regression(self):
            
        throttleSetpoints = self.powerSettingCurrent[:4].reshape(-1,1)
        
        features = PolynomialFeatures(degree=3).fit_transform(self.powerSettingCurrent[:4].reshape(-1,1))
        currentRegression = LinearRegression().fit(features, self.currentMultimeter[:4])
        currentCurve = currentRegression.predict(PolynomialFeatures(degree=3).fit_transform(self.throttleRange))
    
        return currentCurve
             

In [ ]:
ast = Motor(1)

In [ ]:
# get data for all 8 motors

motor = []
for i in range(1,9):
    motor.append(Motor(i))

# motor order
motorOrderVoltage = [1, 6, 5, 2, 8, 4, 7, 3]
motorOrderCurrent = [1, 2, 5, 6, 4, 8, 3, 7]

# manufacturers motor data
manufacturerThrottle = np.array([0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1])*100
manufacturerRPM = np.array([1680, 2280, 2820, 3480, 3900, 4380, 4800])
manufacturerCurrent = np.array([1.8, 3.9, 7.7, 13.7, 19.7, 29.2, 38.8])

# perform linear regression 

throttle = np.linspace(20, 100, 100).reshape(-1,1)

# quadratic regression for rpm
rpmFeatures = PolynomialFeatures(degree=2).fit_transform(manufacturerThrottle.reshape(-1,1))    
rpmRegression = LinearRegression().fit(rpmFeatures, manufacturerRPM)
rpmCurve = rpmRegression.predict(PolynomialFeatures(degree=2).fit_transform(throttle))

# cubic regression for current
currentFeatures = PolynomialFeatures(degree=2).fit_transform(manufacturerThrottle.reshape(-1,1))
currentRegression = LinearRegression().fit(currentFeatures, manufacturerCurrent)
currentCurve = currentRegression.predict(PolynomialFeatures(degree=2).fit_transform(throttle))

# save 
np.currentBiasValues = np.zeros(8)

In [ ]:
# analyse difference between voltage from multimeter and voltage from from ESC
for i in range(8):
    print(motor[i].voltageMultimeter[0])

In [ ]:
# Show voltage measurements for all motors

n_motors = 8
fig0, ax0 = plt.subplots(8,2, figsize=(10,32))

# raw data
for i in range(8):

    m = motorOrderVoltage[i]-1
        
    ax0[i][0].scatter(motor[m].timeVoltage, motor[m].omegaVoltage, color='lightgray', label='RPM raw')
    ax0[i][1].scatter(motor[m].timeVoltage, motor[m].voltage, color='lightgray', label='esc-voltage raw')

    # plot the sequences 

    for j, seq in enumerate(motor[m].voltageSequences):

        time = motor[m].timeVoltage[seq]
        omega =  motor[m].omegaVoltage[seq]
        voltage = motor[m].voltage[seq]
        t = (max(time) + min(time))/2
        mean_voltage = np.mean(voltage)# - motor[m].voltageBias

        if(j==0):
            ax0[i][0].scatter(time, omega, color='deepskyblue', label = 'RPM filtered')
            ax0[i][1].scatter(time, voltage, color='deepskyblue', label='esc-voltage filtered')
            ax0[i][1].scatter(t, mean_voltage, s=50, marker='x', color='b', label='esc-voltage mean')
            ax0[i][1].scatter(t, motor[m].voltageMultimeter[j], s=50, marker='x', color='r',label='multimeter-voltage')
        else:
            ax0[i][0].scatter(time, omega, color='deepskyblue')
            ax0[i][1].scatter(time, voltage, color='deepskyblue')
            ax0[i][1].scatter(t, mean_voltage, s=50, marker='x', color='b')
            ax0[i][1].scatter(t, motor[m].voltageMultimeter[j], s=50, marker='x', color='r')
    
    ax0[i][0].set_xlabel('Time [s]')
    ax0[i][0].set_ylabel('$RPM$')
    ax0[i][0].set_ylim(0,5000)
    ax0[i][0].grid()
    ax0[i][0].legend()

    ax0[i][1].set_xlabel('Time [s]')
    ax0[i][1].set_ylabel('Voltage')
    ax0[i][1].set_ylim(29.5,33.5)
    ax0[i][1].grid()
    ax0[i][1].legend()

    ax0[i][0].set_title(f'Motor {m+1}: RPM over time')
    ax0[i][1].set_title(f'Motor {m+1}: Voltage over time')

fig0.suptitle('Voltage measurements', y=0.9925, fontsize=16)

fig0.tight_layout()

In [ ]:
# Show current measurements for all motors

fig1, ax1 = plt.subplots(8, 2, figsize=(10,32))


# raw data
for i in range(8):
    
    m = motorOrderCurrent[i]-1
    
    ax1[i][0].scatter(motor[m].timeCurrent, motor[m].omegaCurrent, color='lightgray', label='RPM raw')
    ax1[i][1].scatter(motor[m].timeCurrent, motor[m].current, color='lightgray', label='esc-voltage raw')

    # plot the sequences 

    for j, seq in enumerate(motor[m].currentSequences):

        time = motor[m].timeCurrent[seq]
        omega =  motor[m].omegaCurrent[seq]
        current = motor[m].current[seq]# - motor[m].currentBias
        t = (max(time) + min(time))/2
        mean_current = np.mean(current)

        if(j==0):
            ax1[i][0].scatter(time, omega, color='deepskyblue', label = 'RPM filtered')
            ax1[i][1].scatter(time, current, color='deepskyblue', label='esc-voltage filtered')
            ax1[i][1].scatter(t, mean_current, s=50, marker='x', color='b', label='esc-voltage mean')
            ax1[i][1].scatter(t, motor[m].currentMultimeter[j], s=50, marker='x', color='r',label='multimeter-voltage')
        else:
            ax1[i][0].scatter(time, omega, color='deepskyblue')
            ax1[i][1].scatter(time, current, color='deepskyblue')
            ax1[i][1].scatter(t, mean_current, s=50, marker='x', color='b')
            ax1[i][1].scatter(t, motor[m].currentMultimeter[j], s=50, marker='x', color='r')
    
    ax1[i][0].set_xlabel('Time [s]')
    ax1[i][0].set_ylabel('$RPM$')
    ax1[i][0].set_ylim(0,5000)
    ax1[i][0].grid()
    ax1[i][0].legend()

    ax1[i][1].set_xlabel('Time [s]')
    ax1[i][1].set_ylabel('Current')
    #ax1[i][1].set_ylim(25,35)
    ax1[i][1].grid()
    ax1[i][1].legend()

    #ax0[i][0].set_ylim(0,4500)
    #ax0[1][0].set_ylim(0,4500)

    ax1[i][0].set_title(f'Motor {i+1}: RPM over time')
    ax1[i][1].set_title(f'Motor {i+1}: Current over time')
    #ax0[1][0].set_title(f'RPM over time')
    #ax0[1][1].set_title(f'Current over time')

    
fig1.suptitle('Current measurements', y=0.9925, fontsize=16)

fig1.tight_layout()

In [ ]:
# prepare regression for RPM data


In [ ]:
# Map filtered sequences to power settings

# Show voltage measurements for all motors

n_motors = 8
fig0, ax0 = plt.subplots(8,2, figsize=(10,32))

# raw data
for i in range(8):
    
    m = motorOrderVoltage[i]-1

    # plot the sequences        
    for j, seq in enumerate(motor[m].voltageSequences):

        # get values for time (x-axis) and RPM/Voltage (y-axis)
        time = motor[m].timeVoltage[seq]
        omega =  motor[m].omegaVoltage[seq]
        voltage = motor[m].voltage[seq]
        
        # get values for the percentage of max. power that was set during the calibration test
        powerSetting = motor[m].powerSettingVoltage[j]
        powerSettingVector = np.ones(omega.size)* powerSetting
        
        # calculate mean voltage and RPM for every filtered sequence
        mean_voltage = np.mean(voltage)
        mean_omega = np.mean(omega)
        

        if(j==0):
            ax0[i][0].scatter(powerSettingVector, omega, color='deepskyblue')
            ax0[i][0].scatter(powerSetting, mean_omega, s=50, marker='x', color='b', label='RPM measured')
            
            ax0[i][1].scatter(powerSettingVector, voltage, color='deepskyblue', label='esc-voltage filtered')
            ax0[i][1].scatter(powerSetting, mean_voltage, s=50, marker='x', color='b', label='esc-voltage mean')
            ax0[i][1].scatter(powerSetting, motor[m].voltageMultimeter[j], s=50,
                              marker='x', color='r',label='multimeter-voltage')
        else:
            ax0[i][0].scatter(powerSettingVector, omega, color='deepskyblue')
            ax0[i][0].scatter(powerSetting, mean_omega, s=50, marker='x', color='b')
            
            ax0[i][1].scatter(powerSettingVector, voltage, color='deepskyblue')
            ax0[i][1].scatter(powerSetting, mean_voltage, s=50, marker='x', color='b')
            ax0[i][1].scatter(powerSetting, motor[m].voltageMultimeter[j], s=50, marker='x', color='r')
    
    # manufacturer RPM data points + regression curve
    ax0[i][0].scatter(manufacturerThrottle, manufacturerRPM, color='black', marker='x', label='RPM manufacturer')
    ax0[i][0].plot(throttle, rpmCurve, color='black', linewidth=1, linestyle='dashed')#, label='rpm regression')
    
    # regression curve for motor RPM
    ax0[i][0].plot(motor[m].throttleRange, motor[m].omegaRegressionCurve,
                   color='deepskyblue', linestyle='dashed', linewidth=1)

    # labels, title, grid, legend
    ax0[i][0].set_xlabel('Power [%]')
    ax0[i][0].set_ylabel('$RPM$')
    ax0[i][0].set_ylim(0,5000)
    ax0[i][0].grid()
    ax0[i][0].legend()

    ax0[i][1].set_xlabel('Power [%]')
    ax0[i][1].set_ylabel('Voltage')
    ax0[i][1].set_ylim(25,35)
    ax0[i][1].grid()
    ax0[i][1].legend()

    ax0[i][0].set_title(f'Motor {m+1}: RPM over time')
    ax0[i][1].set_title(f'Motor {m+1}: Voltage over time')

fig0.suptitle('Voltage measurements', y=0.9925, fontsize=16)

fig0.tight_layout()

In [ ]:
# Show current measurements for all motors

fig1, ax1 = plt.subplots(8, 2, figsize=(10,32))


# raw data
for i in range(8):
    
    m = motorOrderVoltage[i]-1

    # plot the sequences 

    for j, seq in enumerate(motor[m].currentSequences):
        
        # get values for time (x-axis) and RPM/Voltage (y-axis)
        time = motor[m].timeCurrent[seq]
        omega =  motor[m].omegaCurrent[seq]
        current = motor[m].current[seq]
        
        powerSetting = motor[m].powerSettingCurrent[j]
        powerSettingVector = np.ones(omega.size)* powerSetting
        
        mean_current = np.mean(current)
        mean_omega = np.mean(omega)
        
        if(j==0):
            ax1[i][0].scatter(powerSettingVector, omega, color='deepskyblue', label = 'RPM filtered')
            ax1[i][0].scatter(powerSetting, mean_omega, s=50, marker='x', color='b', label='RPM measured')
            
            ax1[i][1].scatter(powerSettingVector, current, color='deepskyblue')#, label='esc-voltage filtered')
            ax1[i][1].scatter(powerSetting, mean_current, s=50, marker='x', color='b', label='esc-voltage mean')
            ax1[i][1].scatter(powerSetting, motor[m].currentMultimeter[j], s=50, marker='x', color='r',label='multimeter-voltage')
        else:
            ax1[i][0].scatter(powerSettingVector, omega, color='deepskyblue')
            ax1[i][0].scatter(powerSetting, mean_omega, s=50, marker='x', color='b')
            
            ax1[i][1].scatter(powerSettingVector, current, color='deepskyblue')
            ax1[i][1].scatter(powerSetting, mean_current, s=50, marker='x', color='b')
            ax1[i][1].scatter(powerSetting, motor[m].currentMultimeter[j], s=50, marker='x', color='r')
    
    # regression curve for motor RPM
    ax1[i][0].plot(motor[m].throttleRange, motor[m].omegaRegressionCurve,
                   color='deepskyblue', linestyle='dashed', linewidth=1, label='voltage RPM curve')
    
    # manufacturer data + regression curve for current
    ax1[i][1].plot(throttle, currentCurve, color='black', linewidth=1, linestyle='dashed')#, label='rpm regression')
    ax1[i][1].scatter(manufacturerThrottle, manufacturerCurrent, color='black', marker='x', label='manufacturer RPM')
    
    # regression curve for current
    ax1[i][1].plot(motor[m].throttleRange, motor[m].ESCcurrentRegressionCurve,
                   color='deepskyblue', linestyle='dashed', linewidth=1)
    
    # regression curve for current
    #ax1[i][1].plot(motor[m].throttleRange, motor[m].multimeterCurrentRegressionCurve,
    #               color='red', linestyle='dashed', linewidth=1)
    
    ax1[i][0].set_xlabel('Power [%]')
    ax1[i][0].set_ylabel('$RPM$')
    ax1[i][0].set_ylim(0,5000)
    ax1[i][0].grid()
    ax1[i][0].legend()

    ax1[i][1].set_xlabel('Power [%]')
    ax1[i][1].set_ylabel('Current')
    #ax1[i][1].set_ylim(25,35)
    ax1[i][1].grid()
    ax1[i][1].legend()

    #ax0[i][0].set_ylim(0,4500)
    #ax0[1][0].set_ylim(0,4500)

    ax1[i][0].set_title(f'Motor {m+1}: RPM over time')
    ax1[i][1].set_title(f'Motor {m+1}: Current over time')
    #ax0[1][0].set_title(f'RPM over time')
    #ax0[1][1].set_title(f'Current over time')

    
fig1.suptitle('Current measurements', y=0.9925, fontsize=16)

fig1.tight_layout()